In [1]:
import os
import sys
sys.path.append(r"D:\往期\奇货居\ZERO")

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pymysql
from sqlalchemy import create_engine

from tools import *


In [2]:
file_path = r"D:\奇货居\用户模拟数据.xlsx"

df = pd.read_excel(file_path)
data = df.set_index('id')
data['二级佣金'] = 0
data['三级佣金'] = 0
data['无限极佣金'] = 0

data['level'] = 1

commission_dic = {
    "VIP":20,
    "VVIP":50
}

In [3]:
data[:2]

,user_name,parent_name,parent_id,state,一级佣金,二级佣金,无限极佣金,level
id,,,,,,,,
1,A1,root,0,合伙人,0,0,0,1
2,A2,root,0,VVIP,0,0,0,1


In [4]:
data.loc[[1,2],['user_name','parent_name']]

,user_name,parent_name
id,,
1,A1,root
2,A2,root


In [5]:
for index,row in data.iterrows():
    parent_id = row['parent_id']
    state = row['state']
    commission = commission_dic.get(state,0)
    

    
    if parent_id:

        data.loc[index,'level'] += data.loc[parent_id,'level']
        
        data.loc[parent_id,'二级佣金'] +=  commission
        data.loc[parent_id,'无限极佣金'] +=  commission
        
        parent_id = data.loc[parent_id,'parent_id']
        
    if parent_id:
        data.loc[parent_id,'三级佣金'] += commission
        data.loc[parent_id,'无限极佣金'] +=  commission
        parent_id = data.loc[parent_id,'parent_id']

    while parent_id:
        data.loc[parent_id,'无限极佣金'] +=  commission
        parent_id = data.loc[parent_id,'parent_id']
        

In [6]:
data


,user_name,parent_name,parent_id,state,一级佣金,二级佣金,无限极佣金,level
id,,,,,,,,
1,A1,root,0,合伙人,170,190,480,1
2,A2,root,0,VVIP,150,220,490,1
3,B1,A1,1,VVIP,20,0,20,2
4,B2,A1,1,VVIP,70,70,140,2
5,B3,A1,1,VVIP,100,50,150,2
6,B4,A1,1,VIP,0,0,0,2
7,B5,A2,2,VVIP,20,0,20,2
8,B6,A2,2,VVIP,100,70,170,2
9,B7,A2,2,VVIP,100,50,150,2
